# High-Performance Python
Once in awhile, you will hear that
> Oh, Python is so slow or not performant.

> Python is not for <i>enterprise</i> use cases since it is not fast enough.

> Python is a language for script kiddies and at best "glue" code.

ZOMG!
- Scala's functional API is awesome and stands on the shoulders of the JVM. 
- Go(lang) has goroutines that beats the pants off of Python's multithreading. 
- Rust...I don't know what it does but it sounds trendy. 🙄

Spare me your faux concern ~~you pretentious C++ developer~~...I mean good question!

Python supports rapid speed of development and experimentation, expressiveness and conciseness of syntax, and high-performance frameworks for speed. 

At a high-level
- On one machine, you can speed up Python performance using vectorization, multi-threading, multi-processing, and asynchronous programming.
- On multiple machines, you have cluster/distributed computing frameworks: Apache Spark, Dask, Apache Beam, Ray, and others.
- On cloud services, you have containers and scalable, serverless computing.

In this tutorial, I will show you various ways to make Python fast and scalable so that performance is not your bottleneck.

## Debunking the "Slow" Python Myth
What do you mean by slow? Do you mean that Python runs slow, that it would cost more money when renting EC2s?

Remember that compute time is cheap and developer time is relatively expensive. 

Suppose a standard laptop has 4 CPUs and 16 GB of RAM. The equivalent EC2 instance on AWS rents for just $0.154 per hour; an engineer costs $50 per hour. So for each hour of developer time you waste, you could have rented a machine for 300 hours--or you can rent 300 machines for 1 hour. A 40 hour week for on EC2 would be \$6--cheaper than your daily lunch!

Some "faster" languages are notoriously verbose, so you end up with pages and pages of code. Python's ease of use and expressivenes gives you a super power: rapid speed of development, which gives you a second super power: the ability to beat your competition to the market. Way of the Pro! 

Rapid experimentation allows you to out-maneuver your competition with better and newer features. Python's readability offers maintainability since developers come and go. Basically, can you write your code faster than somebody else: can you do something in 1 hour that somebody else takes 5?  

If you need speed, Guido recommends you delegate the performance-critical part to Cython:  

>"At some point, you end up with one little piece of your system, as a whole, where you end up spending all your time. If you write that just as a sort of simple-minded Python loop, at some point you will see that that is the bottleneck in your system. 
>
>It is usually much more effective to take that one piece and replace that one function or module with a little bit of code you wrote in C or C++ rather than rewriting your entire system in a faster language, because for most of what you're doing, the speed of the language is irrelevant." --Guido van Rossum

I won't mention Cython in the tutorial since it is rarely ever used--I have never personally met anybody who wrote their custom Cython code. Things that need to be performant in Python are already written in Cython: numpy, XGBoost, SpaCy, etc. The high level idea for Cython is that Python is actually written on top of C, so for speed you can write directly in C (or C++) and import it to Python.  

If you are concerned about speed, do you know what is faster than Python? Fortran.  Then you can brush up on your Fortran to get hired by ... nobody (or worse, mainframe specialists 🤢).  

Speed of the language primarily helps with CPU-bound problems and not IO-bound problems--in IO-bound problems, any programming language will perform roughly the same since the majority of the time, you are waiting for a network response.  

Finally, you don't really care about the raw speed of a language. You care about the expressioness and functionality of the language: its ecosystem, user-base, mindshare.

Whatever problem you can think of, Python probably has a library for that. Which you can `pip install`. So you can write minimal code. With which you solve your problem. So you can call it a day.

Like Newton, when you use Python, you stand on the shoulders of giants. Things just work! Imagine if you couldn't just import `sklearn` and run Random Forest but had to write your own random forest algorithm. Actually, don't...

Here are some libraries just for scientific computing.  
<p align="center"><img src="images/scientific-ecosystem.png" width=600></p><br>  

In [1]:
import antigravity  # a cute Easter egg; things in Python are so easy. Python has "batteries included"

## The Big O, Not Just Japanese Batman
<p align="center"><img src="images/The-Big-O.jpg" width=300></p>

Data structures and algorithms (i.e. your functions) have different performance characteristics that can be measured by how much memory (space) it takes to do something and how many steps (time) it takes to do something. Fancy people use the terms <i>space and (run)time complexity</i>, respectively. Don't be intimidated by the fancy names; I assure you it's a simple idea.

__Big O notation__ is used to denote the worst-case scenario of how resource utilization (memory or number of steps) will grow with respect to the number of elements in your input data.

__Note:__ O(n) is pronounced oh-enn.

### Time Complexity of Data Structures
Let's walk through some simple examples.  
* list:
    * appending to the end of a list is an (amoritized) O(1) operation. It is a fast operation.
    * appending to the beginning of a list takes O(n) operations because a new list is created, new element is inserted, and all the elements from the original list is copied over. It is slow compared appending to the end of the list.
    * checking if an element is in a list (i.e. a test for membership) takes O(n) operations since the worst case scenario is that  element is not in the list, so you have to check every element of the list.
    * Getting an element at a known, specific ith index is an O(1) operation. Fast!
* dict:
    * checking if a key is in a dictionary takes O(1) operation because dict keys are hashable (and unique).
    * checking if a value is in a dictionary takes O(n) operations because dict values are not hashable (and do not have to be unique).

Ideally all your code takes as little memory and as few steps as possible (i.e. O(1) for instantenous results). In practice, code uses nontrivial amounts of memory and computation. Sometimes, you can consider a tradeoff between space and time requirements. 

For example, if you are using a list and often have to append to the front and the end of the list, then you are better off with a data structure called __deque__, a double-ended queue. A deque has rapid `append()` and `appendleft()` methods as well as `pop()` and `popleft()`, since these operations are all O(1). However, a deque has slow access to the ith element, which is now a O(n) operation. You have to figure out which data structure gives you the best benefit, given its tradeoffs/constraints.

### Time Complexity of Algorithms
As with data structures, algorithms also have space and time complexity. An algorithm is just a fancy word for a set of steps to get you to the right answer: sorting algorithm, sudoku solver algorithm, determining the winner in a tournament algorithm.

When an engineer says something is *(computationally) cheap*, that is fancy lingo for if the calculation uses little resources (low space complexity) and is fast (low time complexity). If a calculation is *(computationally) expensive*, it means it takes a lot of memory or is slow. Ideally you want your algorithm to be cheap.

There are orders of magnitudes for complexity of algorithms (and data structures):
* `O(1)` (i.e. constant time): the best (that you can possibly do)!
* `O(log(n))` (i.e. logarithmic time): good
* `O(n)` (i.e. linear time): still good
* `O(n*log(n))`: fair, use sparingly
* `O(n^2)` (i.e. quadratic time): bad, use very sparingly. If the exponent is any number other than 2 (ie 3, 4, ...), then it is called polynomial time
* `O(2^n)` (i.e. exponential time) and `O(n!)`: horrible, ideally you never have use an algorithm (or data structure) that has these complexities because it will be slow and resource intensive.
    
Basically, you want your algorithm (or data structure) to *scale* well with the size of your input data. To visualize how important the scaling factor, Big O, is to your algorithm, here's a chart. Stay in the green.  
The chart and a more elaborate explanation of Big O is available at [Understanding time complexity with Python examples](https://towardsdatascience.com/understanding-time-complexity-with-python-examples-2bda6e8158a7)
<p align="center"><img src="images/Big_O_magnitudes.png" width=800></p><br>

Here is simple algorithm that concatenates strings together: it exhibits quadratic, O(n^2) time complexity.

In [2]:
from string import ascii_letters
from random import choices

print(ascii_letters)
lots_of_letters = choices(ascii_letters, k=100000)
print(lots_of_letters[:10])

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
['B', 'O', 'H', 'R', 'H', 'o', 'd', 'O', 'G', 'L']


In [3]:
%%time
# functional style: quadratic time
from functools import reduce

result_FP = reduce(lambda accumulated_string, char: accumulated_string + char, lots_of_letters)

CPU times: total: 109 ms
Wall time: 118 ms


In [4]:
%%time
# procedural style: quadratic time
result_procedural = ""
for char in lots_of_letters:
    result_procedural += char

CPU times: total: 109 ms
Wall time: 120 ms


Here's a smarter option that gives you linear, O(n) time complexity.

In [5]:
%%time
result_OOP = "".join(lots_of_letters)  # OOP is much faster since runtime is O(n)

CPU times: total: 0 ns
Wall time: 1e+03 µs


In [6]:
result_FP == result_procedural == result_OOP  # proof that the results are the same

True

"Smart data structures and dumb code works a lot better than the other way around." -from *The Cathedral and the Bazaar* by Eric S. Raymond

When possible, pick a good data structure that fits your problem--because there are a lot of data structures that do cool and useful things. Using the right data structure is the low-hanging fruit that requires minimum effort. If your logic is fundamentally complicated, then you will have to get creative and write (and optimize) a smart algorithm--that is more work. Of course, the quote is simply a rule of thumb--right most of the time, but not all the time. Use good judgment.

Let me give you a practical example. Often in technical interview questions, the interviewer will ask you to write/whiteboard an algorithm. This is an interview question I received many years ago when I was a beginner: "Given a list of words, find which words are anagrams of each other." Here is my very crude solution.

In [7]:
from collections import Counter

def find_anagrams(list_of_words):
    anagrams = {}
    for i, current_word in enumerate(list_of_words):
        for following_word in list_of_words[i+1:]:
            if Counter(current_word) == Counter(following_word):
                if current_word in anagrams:
                    anagrams[current_word].add(following_word)
                else:
                    anagrams[current_word] = {following_word}
    return anagrams

find_anagrams(["rats", "arts", "star", "god", "dog", "apple"])

{'rats': {'arts', 'star'}, 'arts': {'star'}, 'god': {'dog'}}

My algorithm is very inefficient since it has O(n^2) time complexity: it has a double for-loop. And it is duplicating the letter counts (ie calling *Counter()*) multiple times for the same word. Finally, the output isn't great.

The correct way (Way of the Pro) is to find some way to transform the data into what I call a "canonical form" or "canonical representation" of the data that is invariant to differences that don't matter. For example, in this anagram problem, it does not matter what order the letters are in the word. Hence, to get the "canonical form", you sort the letters in the word.

In [8]:
from collections import defaultdict

def find_anagrams_optimized(list_of_words):
    anagrams = defaultdict(list)
    for word in list_of_words:
        word_with_letters_sorted = "".join(sorted(word))
        anagrams[word_with_letters_sorted].append(word)
    return [val for val in anagrams.values() if len(val) > 1]

find_anagrams_optimized(["rats", "arts", "star", "god", "dog", "apple"])

[['rats', 'arts', 'star'], ['god', 'dog']]

Notice that I used both:
* a better data structure: default dict (with *list* as default) allows easy appending vs a regular dict
* a better algorithm: that has linear O(n) time complexity instead of O(n^2). No redundant, repeated computations

Moreover, the new function uses half the lines of code to get better results, so it is more readable and maintainable.

## Vectorized Calculations with SIMD
vectorization with numpy and pandas: SIMD like R's apply. pd.DataFrame.apply is not vectorized but without the speed performance. It's a convenience function.  

CPU vs GPU: ALU vs FPU  
order of magnitude of speed: CPU, RAM, storage, network: have image  

## Multithreading vs Multiprocessing
Multithreading and multiprocessing is when you hope to speed things up rather than wait for things to execute serially (ie a for loop). 
- Multithreading is when you have multiple threads in one process, and thus the threads have access to the same variables and objects in memory--this is called **concurrency**. 
- Multiprocessing is when you have multiple processes where each Python process/interpreter/session has its own memory and is running independently of each other--this is called true **parallelism**. 

Multithreading is faster to start up (<100 ms), but in most cases the right thing to do is to use multiprocessing. Multithreading is used for **IO-bound** tasks whereas multiprocessing is used for **CPU-bound** tasks.
<table><tr>
    <td> <img src="images/concurrency.jpg" alt="Concurrency" style="width: 500px;"/> </td>
    <td> <img src="images/parallelism.jpg" alt="Parallelism" style="width: 500px;"/> </td>
</tr></table>

[Source for these beautiful visual diagrams](https://archive.ph/Rd2QQ)

### Multithreading
Within a Python process, you can create multiple threads. Python has a Global Interpreter Lock (aptly shortened to GIL) that only allows 1 thread to be run at any given time, so you are switching between threads--this is called **context switching**. So why would you use multithreading instead of regular ol' serial execution (ie for loop)?

<img src="images/multithreaded_process.png" alt="Context Switching Overhead" style="width: 300px;"/>

Multithreading is good for IO-bound tasks. The typical example of IO-bound task is scraping/downloading websites.

Suppose you have 100 urls you want to scrap and each url takes 1 second to scrap. Serially scraping the urls will take 100 seconds. However, if you multithread, IO tasks release the GIL, so you can scrap multiple urls "at the same time." Really, the bulk of the time is waiting for the response of data from the url (as any CPU time is virtually instantaneous), so multithreading can shrink that runtime significantly. 

Remember, you are still just using one CPU. Scraping websites does not take a lot of CPU power. Heavy number crunching is called CPU-bound tasks, and multithreading does not help--and in fact, will be slower than serial execution.

A Python process uses 1 CPU, so each thread uses a **time slice** of the CPU. The thread that acquires the GIL gets to run for a few milliseconds, then is paused, and then releases the GIL. Another thread acquires the GIL, gets to run for a few milliseconds, then is paused, and then releases the GIL, etc. 

In practice, because only 1 thread is executed at any 1 time, multithreading is in some sense running "serially"--2 things are happening after each other, 2 things are NOT happening at exactly the same time. Imagine if you are the CPU and are multi-tasking multiple threads, you can be making a phone call, responding to an email, watching TV. You are switching between those threads very quickly (which are interweaved), so it gives the *impression* that you are doing them "at the same time." However, you are only doing 1 thing at a time.

You don't know which thread is next, you don't know when the current thread is going to be paused, and you don't know from which line of code the next thread will resume. Well, you might think this is a bad idea--and you are correct.
* Newbie: multithreading sounds like a bad idea.  
* Intermediate: multithreading sounds like it could be good idea.  
* Pro: multithreading is definitely a bad idea!

In fact, multithreading is such a bad idea that Mozilla has this sign at their office. If you want to skip this entire section on multithreading, feel free to do so. This section is to edify you on *not* using this technique.
<img src="images/be_this_tall_to_use_multithreading.jpg" alt="Don't use multithreading" style="width: 450px;"/> 

Why is multithreading a bad idea? Multiple threads have access to the same/shared objects in memory, so different threads can modify the shared object. The shared object can be corrupted due to the unexpected order of thread execution--this is called a **race condition**. 

The typical example is a bank transaction. Suppose Alice and Bob are 2 threads that share 1 bank account, and both want to withdraw money "at the same time" (within milliseconds of each other). Here's is the linearized timeline (since only 1 thread is running at any moment in time) of Alice and Bob triggering a race condition and corrupting the state of the shared object: the bank account balance. In blockchain, this is called the double spend problem.
* beginning bank account balance: $100
* Alice: begins action of withdrawing $60
* Alice: bank checks that balance (of $100) >= $60: yes!
* Alice: ATM gives Alice $60
* Bob: begins action of withdrawing $70
* Bob: bank checks that balance (of $100) >= $70: yes!
* Bob: ATM gives Bob $70
* Alice: bank account decrements balance to $40 ($100 - $60)
* Bob: bank account decrements balance to $30 ($100 - $70)
* ending bank account balance: $30 (even though $130 was successfully cashed out and that the bank should have prevented Bob from withdrawing money in the first place due to insufficient funds)


Race conditions can also occur with the opposite problem where if Alice and Bob are both depositing money "at the same time" (basically milliseconds of each other) as multiple threads.
* beginning bank account balance: $100
* Alice: begins action of depositing $60
* Alice: bank acccount gets current balance ($100) and is about to increment
* Bob: begins action of depositing $70
* Bob: bank account gets current balance ($100) and is about to increment
* Bob: deposit is successful. Bank account increments balance to $170 ($100 + $70)
* Alice: deposit is successful. Bank account increments balance to $160 ($100 + $60)
* ending bank account balance: $160 (because it just so happens that Alice's thread finished last)

As you can see, multithreaded actions on a shared object are not "thread-safe", as the actions (i.e. the functions) are not atomic. Thread 1 might do something halfway and pause, and thread 2 can do something else on the same object. Not even functional programming can save you now. This problem would have not occurred if you just did things serially (i.e. a for loop).

The typical example of multithreading triggering a race condition is the bank transaction. Let's do something more fun: tug of war! A deque is double-ended queue--you can think of it as a list with fast O(1) operation for `popleft()`, `pop()` (i.e. `popright()`), `appendleft()`, and `append()` (i.e. `appendright()`). Notice that the deque length does not always stay at 9,999,999 (deque length - 1 for the popped value) and the resulting deque has lost its order.

In [9]:
def popleft_and_appendright(dq, iterations, checkpoint_iterations):
    for i in range(iterations):
        value = dq.popleft()
        if i % checkpoint_iterations == 0:
            print("length of deque: {}".format(len(dq)))
        dq.append(value)
    print("final length of deque: {}".format(len(dq)))

def popright_and_appendleft(dq, iterations, checkpoint_iterations):
    for i in range(iterations):
        value = dq.pop()
        if i % checkpoint_iterations == 0:
            print("length of deque: {}".format(len(dq)))
        dq.appendleft(value)
    print("final length of deque: {}".format(len(dq)))

In [10]:
from collections import deque
from threading import Thread

# double-ended queue (deque) has O(1) operation for popleft/popright, appendleft/appendright
dq = deque(range(10000000))  # this is the shared object

thread1 = Thread(target=popleft_and_appendright, args=(dq, 10000000, 1000000))
thread2 = Thread(target=popright_and_appendleft, args=(dq, 10000000, 1000000))

print("Threads are ready to start!")
thread1.start()
print("Thread 1 is started")
thread2.start()
print("Thread 2 is started")

print("Threads 1 and 2 are probably still running")
thread1.join()
print("Thread 1 is done for sure")
thread2.join()
print("Thread 2 is done for sure")

Threads are ready to start!
length of deque: 9999999
Thread 1 is started
length of deque: 9999998
Thread 2 is started
Threads 1 and 2 are probably still running
length of deque: 9999999
length of deque: 9999999
length of deque: 9999998
length of deque: 9999998
length of deque: 9999998


length of deque: 9999998
length of deque: 9999999
length of deque: 9999999
length of deque: 9999999
length of deque: 9999998
length of deque: 9999999
length of deque: 9999998
length of deque: 9999999
length of deque: 9999998
length of deque: 9999999
length of deque: 9999998
length of deque: 9999999
final length of deque: 9999999
Thread 1 is done for sure
length of deque: 9999999
final length of deque: 10000000
Thread 2 is done for sure


In [11]:
from collections import Counter

print("Are all the elements in order? {}".format(Counter(x == y for x, y in zip(dq, range(len(dq))))))
print("first 10: {}".format(list(dq)[:10]))
print("last 10: {}".format(list(dq)[-10:]))

Are all the elements in order? Counter({True: 9083957, False: 916043})
first 10: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
last 10: [9999990, 9999991, 9999992, 9999993, 9999994, 9999995, 9999996, 9999997, 9999998, 9999999]


Here's another example of the race condition: basically the bank account example. The bank account starts at 0 and should end at 0--but where it ends up, nobody knows! 🤔

Also notice that thread is triggered by calling `thread1.start()`. Before this moment, the thread is not executed--it is just sitting there. When the line `thread1.start()` is run, notice that it is "non-blocking", which means that the thread kind of runs in the "background", so Python can execute the next line before thread1 finishes its work. Just like thread1, `thread2.start()` is non-blocking, so you can run the next line of code (such as `print()`). 

So how do you know when a thread is done running (because theoretically Python could have run the final line of the .py file but the thread is still chugging along)? You can call `thread1.join()`, which *is* a blocking operation--Python will wait until that thread has completed its task. If the thread is already done, then `.join()` will finish very quickly. If the thread is still running, then `.join()` will continue to run/block for an indeterminate amount of time before the next line is run.

In [12]:
def incrementer(lst, iterations, checkpoint_iterations):
    for i in range(iterations):
        lst[0] += 1
        if i % checkpoint_iterations == 0:
            print("lst: {}".format(lst))
    print("final lst: {}".format(lst))

def decrementer(lst, iterations, checkpoint_iterations):
    for i in range(iterations):
        lst[0] -= 1
        if i % checkpoint_iterations == 0:
            print("lst: {}".format(lst))
    print("final lst: {}".format(lst))

In [13]:
lst = [0]  # this is the shared object

thread1 = Thread(target=incrementer, args=(lst, 10000000, 1000000))
thread2 = Thread(target=decrementer, args=(lst, 10000000, 1000000))

thread1.start()
thread2.start()

thread1.join()
thread2.join()

lst: [1]lst: [0]

lst: [-516151]
lst: [-610852]
lst: [-647544]
lst: [-1099607]
lst: [-1437549]
lst: [-1191347]
lst: [-1237481]
lst: [-1387681]
lst: [-1443810]
lst: [-1429337]
lst: [-1710916]
lst: [-1608983]
lst: [-1936812]
lst: [-1931937]
lst: [-2019619]lst: [-2000000]

lst: [-1480467]
final lst: [-1410649]
lst: [-999999]
final lst: [0]


The purpose of this example is to show multithreading causes function calls to be "non-atomic". Multiple threads exist in the same memory and have access to the same variables. 2 (or more) threads compete to access and mutate variables. Suppose the first thread is executing some lines that update a list, that thread can be paused at any time in the function--and you have no idea where. Then a second thread can  be activated and then mutate that same variable. By the time, Python goes back to the first thread, the variable could have already changed.

How to make threads not trip over each other on getting/setting the same variable? You create a lock over a piece of code such that the thread has to acquire the lock before it can run that piece of code. When that piece of code is finished, then the thread can release the lock. 

However, a natural question is that wouldn't the threaded code run effectively sequentially since no 2 threads can run the same code protected by the lock at the same time? Yep, you are correct. Adding locks basically makes multithreaded code run sequentially.

The only redeeming thing about multithreading is that if you run threads that apply the same operation on different elements, then you can use `ThreadPool`, which also can function as a context manager.

In [14]:
from multiprocessing.pool import ThreadPool

lst = [0]  # this is the shared object
thread_pool = ThreadPool(2)  # creating 2 threads
thread_pool.map(
    func=lambda lst: incrementer(lst, 10000000, 1000000),  # hard coded the arguments `iterations` and `checkpoint_iterations`
    iterable=[lst] * 2,  # both elements in the list point to the same object
)
thread_pool.close()  # remember to close the threadpool
print(lst[0])  # final result should be 20000000 but it's not

lst: [1]lst: [2]

lst: [1625847]
lst: [2434413]
lst: [3590299]
lst: [4651934]
lst: [5659089]
lst: [6382990]
lst: [7921455]
lst: [8171460]
lst: [9867464]
lst: [10058422]
lst: [11997715]lst: [12000002]

lst: [13886893]
lst: [14045297]
lst: [15932244]lst: [16000002]

lst: [17915222]
lst: [18059254]
final lst: [19709031]
final lst: [20000000]
20000000


In [15]:
# context manager syntax--better, no manual closing required!
lst = [0]  # this is the shared object
with ThreadPool(2) as thread_pool:
    thread_pool.map(
        func=lambda lst: incrementer(lst, 10000000, 1000000),  # hard coded the arguments `iterations` and `checkpoint_iterations`
        iterable=[lst] * 2,  # both elements in the list point to the same object
)
print(lst[0])  # final result should be 20000000 but it's not

lst: [1]
lst: [86958]
lst: [1947472]lst: [2000002]

lst: [3928203]
lst: [4317000]
lst: [5458435]
lst: [6847031]
lst: [7126303]
lst: [8942822]lst: [9000002]

lst: [10488238]
lst: [11463522]
lst: [12562463]
lst: [13384912]
lst: [14685133]
lst: [15292838]
lst: [16910675]
lst: [17114286]
lst: [18879295]
final lst: [19013395]
final lst: [20000000]
20000000


Finally, threads are *not* scalable. The scheduling of the threads (i.e. which one comes next) is actually done by the OS, so although a couple milliseconds seems fast to humans, it is still slow. Computers operate at the nanosecond level (billions of operations per second). 

Switching between threads is called **context switching**. When there are a lot of threads (say 30), then the OS has to pick which thread out of the 30 runs next. In this case, more time is spent acquiring the GIL than actually doing work. It's like the Hunger Games of threads: may the odds be ever in your favor--in acquiring the GIL. 

In this case, it might be better to use serial execution (for loop) compared to using 30 threads. In practice, acquiring the lock is not a costless operation. 

<img src="images/context_switching_cost.jpg" alt="Context Switching Overhead" style="width: 500px;"/>

So where does that leave us? Threads can mutate the same variables in memory and are non-blocking and may provide some speedup for IO operations. 
- The simple answer is: "Don't use multithreading." 
- The more nuanced answer is: "Don't use multithreading for real, production code--you can use multithreading on simple, toy projects where correctness is not essential."

So why did I show all this *useless* stuff? It was to tee up for the *useful*: multiprocessing and asynchronous programming.

## Multiprocessing

How do you get true parallelism (i.e. multiple things running at the *exact* same time)? You use multiple processes (multiprocessing), so there is no context switching and no problems with the Global Interpreter Lock (GIL). Each (sub)process is basically a separate Python process/kernel (with its own GIL) and can run on a different CPU--this is scheduled by the OS, so you don't need to do it yourself. You can think of multiprocessing as being able to create multiple Python (sub)processes (say 10) with very little code.

Multiprocessing is very good for "embarrassingly parallel" problems, which can be subdivided and worked on separately 
and independently. If you want to see the predicted speedup of using multiprocessing, you can look at [Amdahl's law](https://en.wikipedia.org/wiki/Amdahl%27s_law), which is a formula about how much faster will multiprocessing be over regular ol' serial loops. Multiprocessing is not so good where the results of A depend on results of B, which depend on results of C. For example, multiprocessing is good for running many simulations, but it would not be good for something sequential like withdrawals from a bank account (because you want to know when the bank account runs out of money).

A downside of multiprocessing is that each process has separate memory, not shared memory (which multithreading has). Hence, processes generally don't talk to each other. Communication is still possible through IPC (interprocess communication) or RPC (remote process communication), but these are not (explicitly) used often. The other small downside is the start up time for a new process--it takes on the order of 100 milliseconds whereas a new thread can be started much faster. But 100 milliseconds is only (approximately) a blink of an eye.

At a high level, you can think of multiprocessing as distributed functional programming because you are using `map()` over an iterable and the process pool will send each element of the iterable to a process. The process will apply the function onto the element and then return the result back to you.

#### Sound Impressive While Doing Very Little (i.e. Working Smarter)
I want to share my dirty little secret. I ETLed (extract, transform, load) 150 terabytes of data *without* Spark. To put that in perspective, that is the equivalent of 3,000 HD movies. I used 1 large machine (64 CPUs and 256 GB of RAM). The ETL job was to parse compressed zip files, perform some data transformation, and store into a database. In fact, my Python code was so performant that the machine running Python was only at ~50% CPU utilization, but all the servers running NoSQL (HBase) were all maxed out at 100% CPU utilization. NoSQL is known for its rapid read and write speeds, so it was actually quite surprising that the NoSQL was the bounding constraint and not Python. This is a case study of Python being very performant and doing distributed, parallel computation.


My other dirty secret is that multiprocessing is very easy to write. You can do impressive things with very little code. Don't work harder, work smarter.
 
**Note:** multiprocessing also has the context manager that closes the processes for you.

In [16]:
import multiprocessing

def double(x): return 2 * x

# unfortunately multiprocessing on Windows is buggy, you might need to put your code under if __name__ == "__main__":
pool =  multiprocessing.Pool(processes=4)  # I decide to use 4 Python processes, which can use the full power of 4 CPUs
result = pool.map(func=double, iterable=range(10))
pool.close()  # remember to close your process pool when you are done
result  # result is collected in the corresponding order as the input

In [ ]:
# context manager syntax, preferred syntax as automatically closes process pool regardless of hitting an exception
with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:  # use all my CPUs
    result = pool.map(func=double, iterable=range(10))

result

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

`pool.map()` is a *blocking* method call. Python stays at that line until all the results are completed. However, you can make it non-blocking by changing `pool.map()` to `pool.map_async()`. Just make sure that you get all the results before you close the process pool.

In [ ]:
with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
    async_result = pool.map_async(func=double, iterable=range(10))  # non-blocking so runs instantaneously

print(async_result.ready())
print(async_result.successful())  # closed the process pool before all the results are completed
#print(async_result.get())  # stuck forever as the results are not available

False


ValueError: <multiprocessing.pool.MapResult object at 0x7fb73c3d3730> not ready

In [ ]:
%%time
with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
    async_result = pool.map_async(func=double, iterable=range(10))
    async_result.wait()  # wait until all the results collected
    # async_result.wait(1)  # you can control how long you want to wait for collecting results

print(async_result.ready())
print(async_result.successful())
print(async_result.get())

True
True
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
CPU times: user 0 ns, sys: 78.1 ms, total: 78.1 ms
Wall time: 82.7 ms


In [ ]:
with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
    async_result = pool.map_async(func=double, iterable=range(10))
    print(async_result.get())  # get() is a blocking call but then it's basically equal to pool.map()

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


A common question question is: "How many CPUs to use?" It depends on the problem.
* If your function is **CPU-bound** (ie uses a lot of heavy computation), then you can use as many processes as up to the number of CPUs. The reason is that 1 function on 1 process will use up all the CPU cycles of 1 CPU.
* If your function is **IO-bound** (basically does read/writes and does not use a lot of CPUs), you can use more processes than number of CPUs. The reason is that multiple processes can be run on the same CPU since each process uses little CPU. IO-bound problems, if they are not mission critical, can also be addressed with multithreading (mentioned above) or asynchronous programming (mentioned below).

Some gotchas:
* An additional constraint is RAM. If each process takes a lot of RAM, then together all the processes will take too much RAM and then the OS will terminate the process and you'll get `MemoryError`. Hence you might have to manually fiddle with the number of processes to get maximum performance while not hitting a RAM ceiling.
* `multiprocessing` uses `pickle` to serialize between data and functions from your Python process to each subprocess. Hence objects that are not pickleable can cause trouble (ie lambda functions and generators).

In [ ]:
double = lambda x: x * 2

with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
    async_result = pool.map(func=double, iterable=range(10))

PicklingError: Can't pickle <function <lambda> at 0x7fb73c266a60>: attribute lookup <lambda> on __main__ failed

#### Advanced `multiprocessing` Arguments
Everything above is probably all you need for most use cases. Here are some extra features of multiprocessing in case you want some power features.
* `pool.map(chunksize=SOME_INTEGER_HERE)`: `chunksize` allows how many adjacent elements of the iterable to be sent to 1 process. Suppose you have 4 processes and the iterable is `range(40)` and `chunksize=5`, then elements 1,2,3,4,5 will be sent to 1 process and 6,7,8,9,10 will be sent to another process, etc. By default, chunksize=1.
* `multiprocessing.Pool(maxtasksperchild=SOME_INTEGER_HERE)`: `maxtasksperchild` says how many times each process will run a chunk before that process is terminated and restarted. This is helpful in a long running job if you believe there might be a memory leak where the RAM utilization keeps increasing over time. By setting a small `maxtasksperchild`, then even if there is a small memory leak, the processs will be restarted to replace the previous process, so you won't hit your RAM ceiling. By default, `maxtasksperchild=float("inf")`, so the same processes are used for all the elements of the iterable (ie there are no new processes that replace the old ones).
* `multiprocessing.Pool(initializer=SOME_FUNCTION_HERE)`: `initializer` is a function that runs whenever a process is started up.

In [ ]:
import os
import time

def print_process_id(num):
    time.sleep(num)
    print(num, os.getpid(), flush=True)

with multiprocessing.Pool(processes=3) as pool:
    async_result = pool.map(func=print_process_id, iterable=range(10))  # default chunksize=1, so process id alternates
    print(async_result)  # just a bunch of Nones

0 2132
1 2133
2 2134
3 2132
4 2133
5 2134
6 2132
7 2133
8 2134
9 2132
[None, None, None, None, None, None, None, None, None, None]


In [ ]:
with multiprocessing.Pool(processes=3) as pool:
    async_result = pool.map(func=print_process_id, iterable=range(10), chunksize=2)  # adjacent elements are run on the same process

0 2218
1 2218
2 2219
4 2220
3 2219
6 2218
5 2220
8 2219
7 2218
9 2219


In [ ]:
with multiprocessing.Pool(processes=3, maxtasksperchild=2) as pool:
    async_result = pool.map(func=print_process_id, iterable=range(10))  # elements 6 thru 9 go to new processes

0 2304
1 2305
2 2306
3 2304
4 2305
5 2306
6 2342
7 2351
8 2360
9 2342


In [ ]:
def start_process():
    print("Started process {}".format(os.getpid()), flush=True)

# it is now more visible to see that the process is (terminated and) restarted
with multiprocessing.Pool(processes=3, initializer=start_process, maxtasksperchild=2) as pool:    
    async_result = pool.map(func=print_process_id, iterable=range(10), chunksize=1)

Started process 2394
Started process 2397
0Started process 2402 
2394
1 2397
2 2402
3 2394
Started process 2444
4 2397
Started process 2457
5 2402
Started process 2470
6 2444
7 2457
8 2470
9 2444


#### Multiprocessing Summary
* Multiprocessing gives true parallelism (ie multiple things running at the *exact* same time by running on separate CPUs via separate processes).
* Multiprocessing is great for CPU-bound problem over an iterable. Basically you loop over an iterable and each element is sent to a different process (which is run on a separate CPU), so you save time on getting the final results.

#### Last Thoughts
* I use `multiprocessing`. In Python 3, you have the option of using `multiprocessing.Pool` or `concurrent.futures.ProcessPoolExecutor`--both are basically the same thing.
* Multiprocessing is used to run multiple things on **1** machine. If your job is super-duper intense (due to needing more CPUs or running out of RAM), then there are 2 options:
    * **vertical scaling**: get a larger machine with more CPUs and RAM. Run everything on that 1 larger machine
    * **horizontal scaling**: distributed the computations across multiple machines. Horizontal scaling is what the juicy section on *distributed computing frameworks* is all about (mentioned below)🤩!
* If you have dependency across multiple things, you can use Dask delayed (also mentioned in the *distributed computing frameworks* section).
* If you want to do IO-bound problems (but you already know that multithreading is bad), then you want to use asynchronous programming.

## (unfinished) Distributed Computing Frameworks
In some sense, distributed computing (i.e. *parallel computing* or *high-performance computing*) is glorified multiprocessing. Basically, distributed computing aims to *distribute* workloads horizontally instead of vertically. Horizontally scaling means using lots of small or medium-sized machines to do the work. Vertically scaling means using one very large machine (with lots of RAM and CPU). Horizontal scaling is preferable because medium-sized machines are commodities--very easy to get access to and are cheap. Large machines are very expensive and sometimes an instance that large is not available.

Like multiprocessing, distributed computing works very well with "embarrassingly parallel" problems, which are problems that can be subdivided and worked on separately and independently. Fortunately, distributed computing frameworks can coordinate between the worker machines to aggregate results, which is not an embarrassingly parallel task. Take a look at [Amdahl's law](https://en.wikipedia.org/wiki/Amdahl%27s_law), which gives a formula about predicted speedup vs running a regular loop. A common lesson of distributed computing is that it isn’t about doing one thing exceedingly well; it’s about doing nothing poorly.

In my opinion, distributed computing frameworks are glorified `map`/`filter`/`reduce` operations. Basically you want to chain a bunch of transformations together. If you have just one transformation, you might be better off with just `multiprocessing`. 

In distributed computing, you have a lot of elements (basically a list where the elements are distributed over multiple machines) and then you want to do one thing to it, get the outputted list, and then do another thing to that outputted list. Often the transformations are written using functional programming syntax and applied lazily, so they don't use memory or CPUs until you want to trigger the actual computation. Basically the purpose of the previous notebook (4_Functional_Python.ipynb) was to get you familiar with functional programming syntax, so you can use distributed computing frameworks.

Popular distributed computing frameworks in Python are Apache Spark and Dask. Apache Beam and Ray are interested, but they do not have mainstream adoption--I added them just for fun. 🎉


### (unfinished) MapReduce (old news)
Out-of-core-computation (often mistakenly called out of memory computation) enables large matrix multiplications and querying large databases such as Hive query executed with MapReduce  

A `reduce` operation that is commutative and associative can be partially parallelized using a technique call <i>combiner</i>.  

Map reduce on-premise cannot be scalled up, so EMR on AWS gives you the ability to scale up when needed. Even Spark cluster is apportioned and probably isn't ideal for overallocation.  
show mapreduce stage picture  

### (unfinished) Apache Spark (industry standard 🏆)
If you want to get a high-demand skill to get a new job, learn Spark. I always say: `If you know Spark, you can't get fired!`  

Spark is basically MapReduce in memory. The bottleneck in large scale computation is getting the data to the CPU. RAM is way faster than disk; order of magnitudes, get picture of CPU cache, RAM, disk, network  
Simple Spark  
Spark and Dask: often a functional approach, functoolz syntax translates to Dask bags, FP allows embarrassingly parallel and laziness.  
Don't need Scala for speed  
Weird Java exceptions  
Show Simple-Spark  
Weakness: no ability to do multiple actions. The workaround is to split pipeline into 2 such as word count and character simultaneously and then do groupby.  
DataFrame, RDD
show Spark architecture picture  

### (unfinished) Dask (new framework with growing community adoption 🌱)
Distributed + Task  
`If you learn Dask, then you know Spark. And if you know Spark, you can't get fired!`  
Compare and contrast  
No weird Java exceptions  
Show Dask tutorial (tree aggregation example)  
delayed and futures  
x = x.map_blocks(compress).persist().map_blocks(decompress)  
Can do multiple actions  
DataFrame, array, bag, delayed
show Dask architecture picture (basically the same thing)  

### (unfinished) Apache Beam (theoretically interesting but nobody uses it 😞)
explain why it is cool: unified API between Batch + Stream, auto-scaling, serverless, templates, can do multiple actions  
pcollections, ParDo  
explain its weaknesses: does not support data with schema, no graph algorithms like K-Means, also no concept of action so have to use trick to trigger finality  
Throw picture in: 62 machines running for 30 minutes cost \$2.50.  
<p align="center"><img src="images/Dataflow_pipeline.png" width=1000></p>

Spark, Dask, Beam  
Features:  
* **batch**: first class, first class, first class
* stream: first class, kinda, first class
* **real time Dashboard (for debugging and throughput)**: yes, yes, yes
* **interactive mode/REPL**: yes, yes, yes
* **persist (memory and disk)**: memory+disk, only memory, no
* **with persist, can execute different sections independently**: yes, yes, no
* **can split 1 input to multiple output pipelines**: yes, yes, yes
* **simultaneous actions on multiple output pipelines**: no, yes, yes
* support for branching logic of pipeline: yes, yes, yes
* support for iterative algorithms: yes, yes, no
* **GROUP BY logic**: yes, yes, yes
* **JOIN logic**: yes, yes, yes
* **tagged output pipelines**: no, no, yes
* **combiner/aggregate logic**: yes, yes, yes
* **SQL support**: yes, kinda, no
* distributed numpy array: no, yes, no
* ML algorithms: yes, yes, no
* futures: no, yes, no
* delayed: no, yes, no
* support to address "hot keys": yes, yes, yes
* **shared variables**: broad variable and accumulators, no, idk
* supports logging for debugging: yes, yes, yes
* **supports multiple input/output file types**: yes, yes, yes
* click-based or GUI-based template: no, no, yes
* **auto scaling**: idk, idk, yes

Other support:
* integration: many, growing, primarily only GCP
* pure Python (3): no, yes, yes
* **easy to run/test on your own machine**: yes, yes, yes
* performant local cluster on your machine: idk, yes, no
* speed: Scala (fast), Python (less fast), Python (less fast)
* **documentation**: great, great, lacking
* user base: big, medium, low


Which one to use: Spark is a very hot data engineering skill. I tell my teammates: "if you know Spark, then you can't be fired." Since I work on Dask, I also tell my teammates: "if you know Dask, then you know Spark. If you know Spark, then you can't be fired."  
main problem is RAM vs CPU  
MapReduce for RAM. The better way is to use Spark, that scales horizontally instead of vertically. It is still doing multiprocessing. for RAM


### (unfinished) Ray (and Actor Model)
like a blend of functional and OOP because it is stateful between transformations and dependency. Like Dask futures  
Plasma -> Apache Arrow    
totally novel architecture that overcomes IPC/RPC latency  
show Ray architecture picture  


These things are not built in a vacuum: progress stands on the shoulders of giants. Their histories are intertwined. Spark is basically mapreduce but in memory for the speed. Apache Beam originated from Google's internal Millwheel project and was marketed as true streaming. Spark basically copied Beam's approach. Dask was created originally for distributed arrays because Spark doesn't support it. Ray uses Redis and was probably inspired by Akka (framework in Scala). Dask adopted Ray's actor model. Basically copy ideas that work and dump the rest. A lot of frameworks that don't make the cut and just fade out. Other frameworks: Apache Apex (dead), Apache Gearpump (basically dead), Apache Samza (~~lame~~ only Java), Apache Flink (seems like a Hadoop thing, so basically for legacy applications)  

## (unfinished) Asynchronous Programming/Event-Driven Programming
asyncio is in some sense serial with indirection of a coroutine. asynchronous immplemented in 3 ways: callback (hell from nested structure), futures/promises (looks like [flat] method chaining) and flatter, async/await  
yield from  
yield and return in the same function in a coroutine  
you don't need asyncio to do asynchronous programming  
C10k problem  
basically a queue--asynchronous because not executed immediately. The event loop is the buffer that manages queue  

cooperative vs competitive multitasking  
https://pybay.com/site_media/slides/raymond2017-keynote/async_examples.html

## (unfinished) Virtual Machines vs Containers
VM vs Docker/containerization, multi-tenancy  
serverless computing, scalable/decoupled, notifications and (asynchronous) PubSub connected to AWS Lambda (in some sense distributed but independent multiprocessing)  
world has moved on from hardware -> co-location -> virtualization (rented VM) -> container -> function: https://read.acloud.guru/the-evolution-from-servers-to-functions-21833b576744  
Focus on your business logic and delegate the rest to AWS. Who knows (and who cares) how its really implemented? Are you in the business of creating your private cloud/Hadoop or solving business problems that make $$$?  


## (unfinished) What is the Cloud?
There's an old saying: "Nobody gets fired for buying IBM". That was decades ago. I think the modern version would be "Nobody gets fired for buying AWS". The cloud is so versatile with all the services built-in, that if you are still buying hardware for your own datacenters (without good reasons), you *should* be fired.

One of the clients I worked on is such an old-boys club that they still buy hardware, explain story.

AWS is really supply chain maximization, squeezing out efficiencies from elementary pieces: compute, storage, and networking. With these Lego blocks, they can create managed services that are just a copy of a Apache/open-source software and charge for it. The cloud doesn't cost you money--it saves you money. Overall win-win scenario. Like Costco or Trade Joe model. Show all the logos of services from AWS or GCP. Show the open source version and then AWS service    

Why do you buy a car? Because it is better, faster, and/or cheaper than you can make it yourself. It just "works out of the box." You could build a car yourself; it would be worse functionality, slower to get to market, more expensive build from scratch, and a nightmare to maintain. Just buy it off-the-shelf. Focus on your main problem. Don't roll your own--you must be high if you do. Plus, if something breaks, there's somebody you can ~~complain to~~ ask for help.



## Extra Resources
* [Raymond Hettinger, Keynote on Concurrency, PyBay 2017](https://www.youtube.com/watch?v=9zinZmE3Ogk) and presentation materials on:
    * [multithreading](https://pybay.com/site_media/slides/raymond2017-keynote/threading.html)
    * [multiprocessing](https://pybay.com/site_media/slides/raymond2017-keynote/process.html)
    * [asynchronous](https://pybay.com/site_media/slides/raymond2017-keynote/async_examples.html)
* [Asynchornous programming](https://glyph.twistedmatrix.com/2014/02/unyielding.html) from the creator of Twisted and key influencer to asyncio